In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn

In [2]:
#使用tensorflow自带的工具加载MNIST手写数字集合
mnist = input_data.read_data_sets('/data/mnist', one_hot=True)

Extracting /data/mnist/train-images-idx3-ubyte.gz
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
#查看一下数据维度
mnist.train.images.shape
#查看target维度
mnist.train.labels.shape

(55000, 10)

In [4]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

n_input = 28 # MNIST数据输入(shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # 隐层
n_classes = 10 # 总共类别数 (0-9数字)

In [5]:
def RNN(x, weights, biases):
    # 为了适应RNN，把原始的输入调整成'n_steps'个(batch_size, n_input)的tensors
    x = tf.unstack(x, n_steps, 1)

    # 定义一个lstm cell
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # 获取输出
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # 我们只取RNN的最后一个输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder("float", [None, n_steps, n_input], name='X_placeholder')
    y = tf.placeholder("float", [None, n_classes], name='Y_placeholder')
    
    weights = {    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]), name='Weight')}
    biases = {    'out': tf.Variable(tf.random_normal([n_classes]), name='bias')}
    
    pred = RNN(x, weights, biases)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y), name='loss')
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    init = tf.global_variables_initializer()
    
    

In [ ]:
with tf.Session(graph = graph )as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./graphs/LSTM_MNIST', sess.graph)
    step = 1
    # 小于指定的总迭代轮数的情况下，一直训练
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # 对数据形状调整，编程28个28元素的序列
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # 用optimizer进行优化
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算batch上的准确率
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # 计算batch上的loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    writer.close()
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    

Iter 1280, Minibatch Loss= 1.808339, Training Accuracy= 0.41406
Iter 2560, Minibatch Loss= 1.495256, Training Accuracy= 0.56250
Iter 3840, Minibatch Loss= 1.395385, Training Accuracy= 0.51562
Iter 5120, Minibatch Loss= 1.020464, Training Accuracy= 0.65625
Iter 6400, Minibatch Loss= 0.806921, Training Accuracy= 0.72656
Iter 7680, Minibatch Loss= 0.652273, Training Accuracy= 0.78125
Iter 8960, Minibatch Loss= 0.610314, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 0.718693, Training Accuracy= 0.83594
Iter 11520, Minibatch Loss= 0.493892, Training Accuracy= 0.83594
Iter 12800, Minibatch Loss= 0.502242, Training Accuracy= 0.82812
Iter 14080, Minibatch Loss= 0.554619, Training Accuracy= 0.82031
Iter 15360, Minibatch Loss= 0.373100, Training Accuracy= 0.86719
Iter 16640, Minibatch Loss= 0.389553, Training Accuracy= 0.86719
